In [5]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, TextStreamer
import sentencepiece
from urbandev.nlp import check_tokens_in_dict, classify_text

In [7]:
# model pool
# LinkSoul/Chinese-Llama-2-7b
# ckiplab/gpt2-base-chinese
# 

model_path = "FlagAlpha/Llama2-Chinese-7b-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)


sentence = '將「睡眠呼吸中止症」重新納入身心障礙'
labels = ['交通', '法律與正義', '教育', '金融與經濟', '環境與氣候', '能源', '社會福祉', '軍事與國安']

# run inputs through model and mean-pool over the sequence
# dimension to get sequence-level representations
check_tokens_in_dict(labels, tokenizer)

In [ ]:
from urbandev.utils import load_data_excel

data = load_data_excel("./data/JOIN_iVoting_Proposals_categorized.xlsx")

In [7]:
data.head()

,Index,publishDate,url,title,proposal,benefits&impact,#Votes,MinVotesNecessary,SubmissionDate,Followers,Messages,GA,proposer,Category,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,0,2015-09-10 13:26:19,https://join.gov.tw/idea/detail/25824c17-f141-...,Join 平台應提供匯出資料供批次下載,\n目前 Join 平台為方便機關人員作業，在後台有「打包匯出資料」的功能，但前台沒有開放給...,這是 kiang 在 g0v 提出的想法。\n利益：提供民間備份，並可介接第三方進行全文檢索...,22,250,2015-09-10 17:12:05,1,0,無,au,Social,NaN,NaN,NaN,NaN
1,1,2015-09-10 16:08:10,https://join.gov.tw/idea/detail/75185e90-3a37-...,你是否贊成推動「十八歲投票權」及「二十歲被選舉權」?,鑒於國民年滿十六歲即可工作、納稅，\n年滿十八歲就須負完全的刑事責任並有應考試、服公職的權利...,世代正義是我國民主發展所必須正視的課題，\n若設置過高的年齡門檻形同將年輕世代排除在體制性的...,0,0,2015-09-10 16:11:31,0,0,無,森里蛍一,Law & Justice,NaN,NaN,Categories,NaN
2,2,2015-09-10 19:40:31,https://join.gov.tw/idea/detail/4e658586-2a08-...,你是否贊成將國家撥給政黨的競選費用補助金門檻由3.5%降為3%，並設置10%的上限?,雖然之前國家撥給政黨的競選費用補助金門檻從5%下降至3.5%但仍不夠低，\n以國外案例來說德...,修法調降政黨競選費用補助金門檻有利於小黨發展並可促進多元政黨政治發展，\n稚現行修正後之門檻...,5,250,2015-09-22 00:45:06,1,3,無,森里蛍一,Law & Justice,NaN,NaN,English,Chinese
3,3,2015-09-10 20:45:11,https://join.gov.tw/idea/detail/94b5dca9-57fc-...,都更的建議,國家既然採多數決.為何總是被少數人綁架.都更常因釘子戶造成困擾.個人建議.國家要發展.這個問...,NaN,2,250,2015-09-10 20:49:17,0,0,無,樂與喜,Social,NaN,NaN,Transport,交通
4,4,2015-09-10 21:40:32,https://join.gov.tw/idea/detail/3bdab9bf-d874-...,引進鞭刑,依先進國家如新加坡的刑法引進鞭刑讓重刑犯，強姦犯等重大罪犯得到應得的逞罰,促進社會正義，讓正義得以伸張並幫助這些罪犯記取教訓以及促進受害者人權，進而讓國家進步,184,250,2015-09-10 21:42:48,4,0,無,UFO,Law & Justice,NaN,NaN,Law & Justice,法律與正義


In [8]:
labelsLUT = {"Transport": '交通',"Law & Justice": '法律與正義',"Education": '教育',"Finance": '金融與經濟',
"Environment & Climate": '環境與氣候',"Energy": '能源',"Social": '社會福祉',"Military and Natonal Security": '軍事與國安'}

In [10]:
import numpy as np
sims = []
predictions = []
correct = 0
n=80
total = 0
for index, row in data.iloc[:n].iterrows():
    preds, scores = classify_text(row["title"], labels, tokenizer, model)
    predictions.append(preds)
    sims.append(scores)
    gt = row["Category"]
    #if np.isnan(gt):
    #    continue
    if labels[preds[0]]==labelsLUT[gt.strip()]:
        correct+=1
    total+=1
print(f"Accuracy is {correct/(total)}")


/home/mojoee/Research/11_UrbanDevelopment/03_UrbanDevelopmentTaiwan/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (956 > 512). Running this sequence through the model will result in indexing errors


0 tokens in dict


In [34]:
tokenizer.get_vocab().keys()

dict_keys(['園', 'しています', '##9', '做', '轨', '管', '贡', 'a1', '樁', '##壯', '其', '##谩', '伙', 'cnn', '##縈', '讳', '陇', '##凛', '##陨', '砖', '##脫', '猝', '馈', '##嘶', 'carol', 'gallery', '揽', '##昆', '酥', '480', '##uce', '##狀', '蠛', '##曆', '##搀', '戰', '##芪', '畢', '頻', '##с', '##固', '梳', '蠕', '黎', '##厳', '于', '糕', '##嫩', '##bet', '##茏', '##潺', '##詡', '##贱', '蚤', 'mhz', '##閾', '跻', '##声', '##摇', '##容', '##zon', '##堑', '##患', '##餾', 'etc', 'call', '##寢', '琥', '褻', '##52', '謬', 'cdn', 'ᄎ', 'qa', '##ys', '##ube', '##楚', '##裟', '飒', '馔', '2003', '##ク', '##窺', '##蒿', '##頷', 'comment', '##髖', '痊', '爍', '##逝', '460', '##偕', '##✨', 'john', '##₃', '1908', '荔', '骑', 'qr', '##大', '##穫', '##觑', '1919', '##頹', '猖', '齡', '##ument', '##乐', '##靡', '胶', '屬', '##潢', 'domain', '泡', '４', 'or', '胆', '结', '∥', '径', 'a', '萸', '魚', 'la', '##嘤', '##缭', '##蘚', '##鯊', '攫', '碩', '诱', '440', '袪', '##廟', '己', '##闸', '##酱', '##飞', '丈', '住', '返', 'twice', '均', '么', '財', '諺', '##婺', '##陇', '##如', '##ave', '饈', 'fu', '##察', '##護', '##

In [23]:
data.iloc[index]

Index                                                               81
publishDate                                        2016-03-03 20:16:14
url                  https://join.gov.tw/idea/detail/44d24074-fbbc-...
title                                                                讚
proposal                                                             讚
benefits&impact                                                    NaN
#Votes                                                               0
MinVotesNecessary                                                    0
SubmissionDate                                     2016-03-03 20:16:30
Followers                                                            0
Messages                                                             0
GA                                                                   無
proposer                                                           王念茹
Category                                                           NaN
Unname

In [ ]:
model.dict